In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mate import distribution
from mate.estimate import MaximumLikelihoodTrustEstimator, SIRParticleFilterTrustEstimator

In [ ]:
def plot_estims(estim_mle, estim_pf, trusts):
    plt.hist(estim_pf.sample(10000), density=True, bins=20, alpha=0.5, label="PF")
    plt.hist(estim_mle.dist.rvs(n=10000), density=True, bins=20, alpha=0.5, label="MLE")
    plt.hist(trusts, density=True, bins=20, alpha=0.5, label="Measurements")
    plt.xlim([0,1])
    plt.legend()
    plt.show()

In [ ]:
# configuration for particle filter
n_particles = 10000
prior = {"type": "Beta", "mean": 0.5, "var": 0.1**2}
likelihood = {"type": "Normal", "mean": 0.0, "variance": 0.05**2}
estim_pf = SIRParticleFilterTrustEstimator(
    n_particles=n_particles, prior=prior, likelihood=likelihood, n_steps_between_resampling=0,
)

# configuration for mle estimator
prior = {"type": "Beta", "mean": 0.5, "var": 0.1**2}
estim_mle = MaximumLikelihoodTrustEstimator(
    t0=0.0, distribution=prior, update_rate=1, time_window=10, forgetting=0,
)

# run them
dt = 0.1
n_steps = 100
trusts = []
for i in range(n_steps):
    t = i * dt
    trust = np.clip(0.10 + 0.1**2*np.random.randn(1), 0.0, 1.0)
    trusts.append(trust[0])
    estim_pf.update(timestamp=t, trust=trust)
    estim_mle.update(timestamp=t, trust=trust)

plot_estims(estim_mle, estim_pf, trusts)

In [ ]:
# configuration for particle filter
n_particles = 10000
prior = {"type": "Uniform", "a": 0.0, "b": 1.0}
likelihood = {"type": "Normal", "mean": 0.0, "variance": 0.05**2}
estim_pf = SIRParticleFilterTrustEstimator(
    n_particles=n_particles, n_eff_pct=0.5, prior=prior, likelihood=likelihood,
    n_steps_between_resampling=5, verbose=True,
)

# configuration for mle estimator
prior = {"type": "Beta", "mean": 0.5, "var": 0.1**2}
estim_mle = MaximumLikelihoodTrustEstimator(
    t0=0.0, distribution=prior, update_rate=1, time_window=10, forgetting=0.4,
)

# run them
dt = 0.1
n_steps = 100
trusts = []
for i in range(n_steps):
    t = i * dt
    if i % 2 == 0:
        trust = np.clip(0.1 + 0.1**2*np.random.randn(1), 0.0, 1.0)
        trusts.append(trust[0])
        estim_pf.update(timestamp=t, trust=trust)
        estim_mle.update(timestamp=t, trust=trust)
    else:
        trust = np.clip(0.8 + 0.1**2*np.random.randn(1), 0.0, 1.0)
        trusts.append(trust[0])
        estim_pf.update(timestamp=t, trust=trust)
        estim_mle.update(timestamp=t, trust=trust)

plot_estims(estim_mle, estim_pf, trusts)